In [78]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

In [77]:
df = pd.read_csv('tweet_emotions.csv')
df

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
...,...,...,...
39995,1753918954,neutral,@JohnLloydTaylor
39996,1753919001,love,Happy Mothers Day All my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [80]:
df['content'] = df.content.apply(lambda x: re.sub(r'@[^ ]*',r'',x))


In [134]:
ohe = OneHotEncoder()
labels = ohe.fit_transform(df.sentiment.to_numpy().reshape(-1,1))
# labels = keras.utils.to_categorical(labels.toarray(), num_classes=13)

In [136]:
labels.toarray()[0]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [118]:
texts = df.content.values

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
max_length = max(len(sequence) for sequence in sequences)

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")


In [137]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels.toarray(),test_size=0.25)

In [173]:
model = Sequential()

input_shape = X_train.shape
input_dim = len(tokenizer.word_index) + 1

model = keras.Sequential([
    layers.Embedding(input_dim=input_dim, output_dim=200, input_length=max_length),
    layers.Bidirectional(layers.LSTM(256,activation='relu')),
    layers.Reshape((2,256)),
    layers.SimpleRNN(64,activation='relu'),
    # layers.Bidirectional(layers.LSTM(128,activation='relu')),
    layers.Dropout(0.2),
    layers.Dense(64,activation='relu'),
    layers.Dense(13, activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
# model.build()
model.summary()

Model: "sequential_72"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 36, 200)           6649400   
                                                                 
 bidirectional_22 (Bidirecti  (None, 512)              935936    
 onal)                                                           
                                                                 
 reshape_2 (Reshape)         (None, 2, 256)            0         
                                                                 
 simple_rnn_33 (SimpleRNN)   (None, 64)                20544     
                                                                 
 dropout_17 (Dropout)        (None, 64)                0         
                                                                 
 dense_90 (Dense)            (None, 64)                4160      
                                                     

In [174]:
model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

938/938 [==============================] - 571s 606ms/step - loss: 2.0977 - accuracy: 0.2881 - val_loss: 1.9320 - val_accuracy: 0.3356


In [207]:
sequences = tokenizer.texts_to_sequences(['i feel afraid of what is happening in the world'])

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

prediction = model.predict(padded_sequences)

1/1 [==============================] - 0s 23ms/step


In [208]:
df.sentiment.unique()[np.argmax(prediction)]

'anger'